In [19]:
import json
from urllib.parse import urlencode
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import requests
from lxml import html

import pickle
idx = pd.IndexSlice


In [5]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}

    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(pd.DataFrame(d, columns=h))

In [3]:
def SportsRefCBBRowMaker(player_url,table_name,player_name,correct_year):
    player_id = Player_ID_Dict[player_name]
    
    page = requests.get(player_url,headers=headers)
    tree = html.fromstring(page.content)
    
    if '404' in tree.xpath('//title')[0].text:
        return('404')
    
    else:
        year = tree.xpath('//table/tbody/tr')[-1].items()[0][-1][-4:]
        if(year!=correct_year):
            return(year)
        path = '//div[@id="'+table_name+'"]'
        treeStr = str(tree.xpath(path)[0].getchildren()[-1])[8:-4]
        #This is tricky and comments might be nice here. 

        tree2 = html.fromstring(treeStr)
        
        valuesHTML = tree2.xpath('//table/tfoot/tr')[-1].getchildren()[1:]
        ## This gives career in particular 
        conf = tree2.xpath('//table/tbody/tr/td')[1].getchildren()[0].text
        colNamesHTML = tree2.xpath('//table/thead/tr/th')
        colNames = ["PLAYER_ID","PLAYER_NAME"]+["CONF"] + [col.text for col in colNamesHTML][1:]
        values = [player_id,player_name] + [conf] + [val.text for val in valuesHTML]

        row = pd.DataFrame(pd.Series(values,index=colNames)\
                .dropna()).transpose().set_index("PLAYER_ID")
 #       print(player_url)
        return(row)

def sportsRefCBBTableMaker(player_df, table_name):
    base_url = 'https://www.sports-reference.com/cbb/players/'
    columns_df_url = "https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html"
    columns_df = SportsRefCBBRowMaker(columns_df_url,table_name
                                      ,"Malcolm Brogdon(2016)",'2016')
    CBB_df = pd.DataFrame(columns=columns_df.columns)
    for index,row in dfMerged.iterrows():
        player_name = row['PLAYER_NAME']
        first = re.sub('[^a-zA-Z]+', '',player_name.split()[0].lower())
        last = "".join(player_name.split()[1:])[0:-6].lower().strip(".")
        year = player_name.split()[1][-5:-1]
        name_url = base_url + first + "-" + last + "-"

        i = 1
        stopping_condition=False
        while(stopping_condition==False):
            data_url = name_url + str(i) + ".html"
#            print(data_url)
            row = SportsRefCBBRowMaker(data_url,table_name,player_name,year)
            if isinstance(row, pd.DataFrame):
                stopping_condition=True
                continue
            if(row=='404'):
                stopping_condition=True
                row = None
                if(last[-2:]=="jr"):
                    last = last[:-2]
                    name_url = base_url + first + "-" + last + "-"
                    i=1
                    stopping_condition = False 
            i+=1
        CBB_df = pd.concat([CBB_df,row])
    return(CBB_df)

In [15]:
main_df = pickle.load(open('NBA_2014_18_Adv_100_Score_Def.pkl','rb'))

In [145]:
qualified=main_df.loc[idx[:,:,'Regular Season'],:].query('SEASON_MIN>800')
qualified_and_averaged = qualified.groupby(['PLAYER_ID']).mean()

In [146]:
len(list(qualified.unstack().unstack().index.unique()))

520

Lets look at drafts from 2010 onwards. Then see if these playesr made our list, then if their in college.


In [147]:
draft_history = get_nba_data('drafthistory',{'LeagueID':'00'},)

In [148]:
draft_2010_17 = draft_history[pd.Series(draft_history.SEASON,dtype=int)>2010]

In [149]:
college_players_list= draft_2010_17[draft_2010_17.ORGANIZATION_TYPE=="College/University"].PERSON_ID

In [150]:
qualified_player_list=list(qualified.unstack().unstack().index.unique())

In [157]:
indexes = [str(player) in qualified_player_list for player in college_players_list]

In [163]:
indexes = pd.Series(college_players_list[indexes],dtype=str)

In [192]:
a = qualified.PLAYER_NAME.drop_duplicates()

In [213]:
a.reset_index(level=list(a.index.levels[0]))

KeyError: 'Level 101106 not found'

In [212]:
list(a.index.levels[0])

['101106',
 '101107',
 '101108',
 '101109',
 '101110',
 '101111',
 '101112',
 '101114',
 '101115',
 '101122',
 '101123',
 '101126',
 '101127',
 '101128',
 '101131',
 '101133',
 '101135',
 '101138',
 '101139',
 '101141',
 '101142',
 '101145',
 '101150',
 '101154',
 '101155',
 '101161',
 '101162',
 '101179',
 '101181',
 '101183',
 '101187',
 '101198',
 '101236',
 '101249',
 '1495',
 '1497',
 '1536',
 '1626143',
 '1626144',
 '1626145',
 '1626146',
 '1626147',
 '1626148',
 '1626149',
 '1626150',
 '1626151',
 '1626153',
 '1626154',
 '1626155',
 '1626156',
 '1626157',
 '1626158',
 '1626159',
 '1626161',
 '1626162',
 '1626163',
 '1626164',
 '1626166',
 '1626167',
 '1626168',
 '1626169',
 '1626170',
 '1626171',
 '1626172',
 '1626173',
 '1626174',
 '1626175',
 '1626176',
 '1626177',
 '1626178',
 '1626179',
 '1626181',
 '1626183',
 '1626184',
 '1626185',
 '1626188',
 '1626191',
 '1626192',
 '1626195',
 '1626196',
 '1626199',
 '1626202',
 '1626203',
 '1626204',
 '1626205',
 '1626209',
 '1626210',